In [1]:
import pandas as pd 
from litellm import completion 

df = pd.read_json("new_entries_test_set.jsonl", lines=True, orient="records")


df.head()

,question,answers,subject,wikidata_ID,aliases,questionType,time_interval_entries,temporal_marker_entries
0,List all teams John Nutter played to this day.,"{'Wycombe Wanderers F.C.': '2001', 'Blackburn ...",John Nutter,Q6250736,[],sports,"{""question"": ""Which teams did John Nutter play...","{""question"": ""Which teams did John Nutter play..."
1,List all political positions Brian Pallister ...,"{'22nd Premier of Manitoba': '2016-2021', 'Lea...",Brian Pallister,Q2924976,[Brian William Pallister],political,"{""question"": ""What political positions did Bri...","{""question"": ""What political positions did Bri..."
2,List all political positions Rob Nicholson he...,{'Shadow Minister of Justice Shadow Attorney G...,Rob Nicholson,Q1566929,[Robert Nicholson],political,"{""question"": ""What political positions did Rob...","{""question"": ""Which political positions did Ro..."
3,List all teams Ben May played to this day.,"{'Millwall F.C.': '2001-2008, 2000-2001', 'Sou...",Ben May,Q2895519,"[Benjamin Steven May, Benjamin May]",sports,"{""question"": ""Which teams did Ben May play for...","{""question"": ""Which teams did Ben May play for..."
4,List all teams Steven Caulker played to this ...,"{'Tottenham Hotspur F.C.': '2009-2013', 'Totte...",Steven Caulker,Q26192,[Steven Roy Caulker],sports,"{""question"": ""Which teams did Steven Caulker p...","{""question"": ""Which teams did Steven Caulker p..."


In [31]:
SYS_PROMPT = \
"""## Task: 
Your role is to evaluate synthetically created entries. You will receive an original question-answer pair and a synthetically generated question-answer pair that is intended to be more complex.

Your objective is to determine if the generated question-answer pair is accurate, using the original pair as the baseline for truth. Pay extra attention if the years in the generated answer match the orignal answer.

Respond with:
- '1' if the new question-answer pair is correct
- '0' if the new question-answer pair is incorrect

## Output:
Your output should be a JSON object structured like this:
```json
{
    "reasoning": <str, "Your reasoning for the answer">,
    "is_correct": <int, 1 if the new question-answer pair is correct, 0 if it is incorrect>
}
```
Generate only the JSON object and stop output immediately after.
"""

In [32]:
USR_PROMPT = \
""" ## Original question:
{original_question}
## Original answer:
{original_answer}
## Generated sample:
{generated_sample}
"""



In [33]:
messages_time_interval = []
messages_temporal_markers = []

for i in range(len(df)):
    message1 = []
    message2 = []

    message1.append({"role": "system", "content": SYS_PROMPT})
    message2.append({"role": "system", "content": SYS_PROMPT})

    message1.append({"role": "user", "content": USR_PROMPT.format(original_question=df.iloc[i]["question"], original_answer=df.iloc[i]["answers"], generated_sample=df.iloc[i]["time_interval_entries"])})
    message2.append({"role": "user", "content": USR_PROMPT.format(original_question=df.iloc[i]["question"], original_answer=df.iloc[i]["answers"], generated_sample=df.iloc[i]["temporal_marker_entries"])})

    messages_time_interval.append(message1)
    messages_temporal_markers.append(message2)


len(messages_temporal_markers)





497

In [34]:
messages_time_interval[1]

[{'role': 'system',
  'content': '## Task: \nYour role is to evaluate synthetically created entries. You will receive an original question-answer pair and a synthetically generated question-answer pair that is intended to be more complex.\n\nYour objective is to determine if the generated question-answer pair is accurate, using the original pair as the baseline for truth. Pay extra attention if the years in the generated answer match the orignal answer.\n\nRespond with:\n- \'1\' if the new question-answer pair is correct\n- \'0\' if the new question-answer pair is incorrect\n\n## Output:\nYour output should be a JSON object structured like this:\n```json\n{\n    "reasoning": <str, "Your reasoning for the answer">,\n    "is_correct": <int, 1 if the new question-answer pair is correct, 0 if it is incorrect>\n}\n```\nGenerate only the JSON object and stop output immediately after.\n'},
 {'role': 'user',
  'content': ' ## Original question:\nList all political positions  Brian Pallister he

In [35]:
from litellm import batch_completion

responses_time_interval = batch_completion(model="gpt-4o-2024-11-20", messages=messages_time_interval, temperature=0.1)


responses_time_interval


[ModelResponse(id='chatcmpl-Ad2dZs7FhUE6Udyla1rbdIr4rr0WT', created=1733868293, model='gpt-4o-2024-11-20', object='chat.completion', system_fingerprint='fp_d924043139', choices=[Choices(finish_reason='stop', index=0, message=Message(content='```json\n{\n    "reasoning": "The generated question asks for the teams John Nutter played for between 2006 and 2011. According to the original answer, the teams he played for during this period are \'Stevenage Borough\' (2006-2008), \'Gillingham (loan)\' (2007-2008), and \'Gillingham F.C.\' (2008-2011). These match the answers provided in the generated sample, making it accurate.",\n    "is_correct": 1\n}\n```', role='assistant', tool_calls=None, function_call=None))], usage=Usage(completion_tokens=112, prompt_tokens=474, total_tokens=586, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetai

In [36]:
response_temporal_markers = batch_completion(model="gpt-4o-2024-11-20", messages=messages_temporal_markers, temperature=0.1)


response_temporal_markers


[ModelResponse(id='chatcmpl-Ad2dopZBelTkxhFKoiTcIESv0i9XL', created=1733868308, model='gpt-4o-2024-11-20', object='chat.completion', system_fingerprint='fp_753956bfcf', choices=[Choices(finish_reason='stop', index=0, message=Message(content='```json\n{\n    "reasoning": "The generated question asks for the teams John Nutter played for after leaving Gillingham F.C. According to the original answer, after his time at Gillingham F.C. (2008-2011), he played for Lincoln City F.C. (2011-2013), Woking (loan) (2012-2013), and Woking F.C. (2013-2015). These teams and their respective years match the generated answer, making it accurate.",\n    "is_correct": 1\n}\n```', role='assistant', tool_calls=None, function_call=None))], usage=Usage(completion_tokens=119, prompt_tokens=474, total_tokens=593, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptT

In [40]:
responses_time_interval[0].choices[0].message.content

'```json\n{\n    "reasoning": "The generated question asks for the teams John Nutter played for between 2006 and 2011. According to the original answer, the teams he played for during this period are \'Stevenage Borough\' (2006-2008), \'Gillingham (loan)\' (2007-2008), and \'Gillingham F.C.\' (2008-2011). These match the answers provided in the generated sample, making it accurate.",\n    "is_correct": 1\n}\n```'

In [41]:
response_time_interval_message = [response.choices[0].message.content for response in responses_time_interval]
response_temporal_markers_message = [response.choices[0].message.content for response in response_temporal_markers]


response_time_interval_message



['```json\n{\n    "reasoning": "The generated question asks for the teams John Nutter played for between 2006 and 2011. According to the original answer, the teams he played for during this period are \'Stevenage Borough\' (2006-2008), \'Gillingham (loan)\' (2007-2008), and \'Gillingham F.C.\' (2008-2011). These match the answers provided in the generated sample, making it accurate.",\n    "is_correct": 1\n}\n```',
 '```json\n{\n    "reasoning": "The generated question asks for political positions held by Brian Pallister specifically between 2012 and 2021. The generated answer correctly lists the positions he held during this time frame: \'22nd Premier of Manitoba\' (2016-2021), \'Leader of the Opposition in Manitoba\' (2012-2016), \'Leader of the Progressive Conservative Party of Manitoba\' (2012-2021), and \'Member of the Legislative Assembly of Manitoba for Fort Whyte\' (2012-2021). These align with the original answer, and no positions outside the specified time frame are included.

In [43]:
import json_repair


response_time_interval_message = [json_repair.repair_json(message) for message in response_time_interval_message]
response_temporal_markers_message = [json_repair.repair_json(message) for message in response_temporal_markers_message]


response_time_interval_message



['{"reasoning": "The generated question asks for the teams John Nutter played for between 2006 and 2011. According to the original answer, the teams he played for during this period are \'Stevenage Borough\' (2006-2008), \'Gillingham (loan)\' (2007-2008), and \'Gillingham F.C.\' (2008-2011). These match the answers provided in the generated sample, making it accurate.", "is_correct": 1}',
 '{"reasoning": "The generated question asks for political positions held by Brian Pallister specifically between 2012 and 2021. The generated answer correctly lists the positions he held during this time frame: \'22nd Premier of Manitoba\' (2016-2021), \'Leader of the Opposition in Manitoba\' (2012-2016), \'Leader of the Progressive Conservative Party of Manitoba\' (2012-2021), and \'Member of the Legislative Assembly of Manitoba for Fort Whyte\' (2012-2021). These align with the original answer, and no positions outside the specified time frame are included.", "is_correct": 1}',
 '{"reasoning": "The

In [46]:
import json


response_time_interval_message_decision = [json.loads(message).get("is_correct") for message in response_time_interval_message]
response_temporal_markers_message_decision = [json.loads(message).get("is_correct") for message in response_temporal_markers_message]


response_time_interval_message_decision




[1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [47]:
df.head()

,question,answers,subject,wikidata_ID,aliases,questionType,time_interval_entries,temporal_marker_entries
0,List all teams John Nutter played to this day.,"{'Wycombe Wanderers F.C.': '2001', 'Blackburn ...",John Nutter,Q6250736,[],sports,"{""question"": ""Which teams did John Nutter play...","{""question"": ""Which teams did John Nutter play..."
1,List all political positions Brian Pallister ...,"{'22nd Premier of Manitoba': '2016-2021', 'Lea...",Brian Pallister,Q2924976,[Brian William Pallister],political,"{""question"": ""What political positions did Bri...","{""question"": ""What political positions did Bri..."
2,List all political positions Rob Nicholson he...,{'Shadow Minister of Justice Shadow Attorney G...,Rob Nicholson,Q1566929,[Robert Nicholson],political,"{""question"": ""What political positions did Rob...","{""question"": ""Which political positions did Ro..."
3,List all teams Ben May played to this day.,"{'Millwall F.C.': '2001-2008, 2000-2001', 'Sou...",Ben May,Q2895519,"[Benjamin Steven May, Benjamin May]",sports,"{""question"": ""Which teams did Ben May play for...","{""question"": ""Which teams did Ben May play for..."
4,List all teams Steven Caulker played to this ...,"{'Tottenham Hotspur F.C.': '2009-2013', 'Totte...",Steven Caulker,Q26192,[Steven Roy Caulker],sports,"{""question"": ""Which teams did Steven Caulker p...","{""question"": ""Which teams did Steven Caulker p..."


In [48]:
df['time_interval_filtered'] = response_time_interval_message_decision
df['temporal_marker_filtered'] = response_temporal_markers_message_decision


df.head()



,question,answers,subject,wikidata_ID,aliases,questionType,time_interval_entries,temporal_marker_entries,time_interval_filtered,temporal_marker_filtered
0,List all teams John Nutter played to this day.,"{'Wycombe Wanderers F.C.': '2001', 'Blackburn ...",John Nutter,Q6250736,[],sports,"{""question"": ""Which teams did John Nutter play...","{""question"": ""Which teams did John Nutter play...",1,1
1,List all political positions Brian Pallister ...,"{'22nd Premier of Manitoba': '2016-2021', 'Lea...",Brian Pallister,Q2924976,[Brian William Pallister],political,"{""question"": ""What political positions did Bri...","{""question"": ""What political positions did Bri...",1,1
2,List all political positions Rob Nicholson he...,{'Shadow Minister of Justice Shadow Attorney G...,Rob Nicholson,Q1566929,[Robert Nicholson],political,"{""question"": ""What political positions did Rob...","{""question"": ""Which political positions did Ro...",1,1
3,List all teams Ben May played to this day.,"{'Millwall F.C.': '2001-2008, 2000-2001', 'Sou...",Ben May,Q2895519,"[Benjamin Steven May, Benjamin May]",sports,"{""question"": ""Which teams did Ben May play for...","{""question"": ""Which teams did Ben May play for...",1,1
4,List all teams Steven Caulker played to this ...,"{'Tottenham Hotspur F.C.': '2009-2013', 'Totte...",Steven Caulker,Q26192,[Steven Roy Caulker],sports,"{""question"": ""Which teams did Steven Caulker p...","{""question"": ""Which teams did Steven Caulker p...",1,1


In [50]:
df['time_interval_filtered'].value_counts()


# df['temporal_marker_filtered'].value_counts()




time_interval_filtered
1    423
0     74
Name: count, dtype: int64

In [60]:
filtered_time_interval_df = df[df['time_interval_filtered'] == 1]
filtered_temporal_markers_df = df[df['temporal_marker_filtered'] == 1]



In [61]:
filtered_time_interval_df.drop(columns=['time_interval_filtered', 'temporal_marker_filtered', 'temporal_marker_entries'], inplace=True)
filtered_temporal_markers_df.drop(columns=['time_interval_filtered', 'temporal_marker_filtered', 'time_interval_entries'], inplace=True)


# filtered_time_interval_df.to_json("filtered_time_interval_df.jsonl", orient="records", lines=True)
# filtered_temporal_markers_df.to_json("filtered_temporal_markers_df.jsonl", orient="records", lines=True)




/var/folders/50/hlflxz7x2t9332_cq2tq_1z40000gn/T/ipykernel_51976/2242628008.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_time_interval_df.drop(columns=['time_interval_filtered', 'temporal_marker_filtered', 'temporal_marker_entries'], inplace=True)
/var/folders/50/hlflxz7x2t9332_cq2tq_1z40000gn/T/ipykernel_51976/2242628008.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_temporal_markers_df.drop(columns=['time_interval_filtered', 'temporal_marker_filtered', 'time_interval_entries'], inplace=True)


In [62]:
filtered_time_interval_df

,question,answers,subject,wikidata_ID,aliases,questionType,time_interval_entries
0,List all teams John Nutter played to this day.,"{'Wycombe Wanderers F.C.': '2001', 'Blackburn ...",John Nutter,Q6250736,[],sports,"{""question"": ""Which teams did John Nutter play..."
1,List all political positions Brian Pallister ...,"{'22nd Premier of Manitoba': '2016-2021', 'Lea...",Brian Pallister,Q2924976,[Brian William Pallister],political,"{""question"": ""What political positions did Bri..."
2,List all political positions Rob Nicholson he...,{'Shadow Minister of Justice Shadow Attorney G...,Rob Nicholson,Q1566929,[Robert Nicholson],political,"{""question"": ""What political positions did Rob..."
3,List all teams Ben May played to this day.,"{'Millwall F.C.': '2001-2008, 2000-2001', 'Sou...",Ben May,Q2895519,"[Benjamin Steven May, Benjamin May]",sports,"{""question"": ""Which teams did Ben May play for..."
4,List all teams Steven Caulker played to this ...,"{'Tottenham Hotspur F.C.': '2009-2013', 'Totte...",Steven Caulker,Q26192,[Steven Roy Caulker],sports,"{""question"": ""Which teams did Steven Caulker p..."
...,...,...,...,...,...,...,...
491,List all teams Harrison Afful played to this ...,"{'Feyenoord Academy': '2005-2009, -2005', 'Gha...",Harrison Afful,Q577537,[],sports,"{""question"": ""Which teams did Harrison Afful p..."
493,List all teams Andrés Iniesta played to this ...,"{'FC Barcelona B': '2000-2003', 'Albacete Balo...",Andrés Iniesta,Q43729,[Andrés Iniesta Luján],sports,"{""question"": ""Which teams did Andr\u00e9s Inie..."
494,List all teams Darren Bent played to this day.,"{'Ipswich Town F.C.': '2001-2005, 1998-2001', ...",Darren Bent,Q213459,[Darren Ashley Bent],sports,"{""question"": ""List all teams Darren Bent playe..."
495,List all teams Robert Green played to this day.,"{'Norwich City F.C.': '1999-2006, 1992-1999', ...",Robert Green,Q200770,[Robert Paul Green],sports,"{""question"": ""Which teams did Robert Green pla..."


In [63]:
filtered_temporal_markers_df

,question,answers,subject,wikidata_ID,aliases,questionType,temporal_marker_entries
0,List all teams John Nutter played to this day.,"{'Wycombe Wanderers F.C.': '2001', 'Blackburn ...",John Nutter,Q6250736,[],sports,"{""question"": ""Which teams did John Nutter play..."
1,List all political positions Brian Pallister ...,"{'22nd Premier of Manitoba': '2016-2021', 'Lea...",Brian Pallister,Q2924976,[Brian William Pallister],political,"{""question"": ""What political positions did Bri..."
2,List all political positions Rob Nicholson he...,{'Shadow Minister of Justice Shadow Attorney G...,Rob Nicholson,Q1566929,[Robert Nicholson],political,"{""question"": ""Which political positions did Ro..."
3,List all teams Ben May played to this day.,"{'Millwall F.C.': '2001-2008, 2000-2001', 'Sou...",Ben May,Q2895519,"[Benjamin Steven May, Benjamin May]",sports,"{""question"": ""Which teams did Ben May play for..."
4,List all teams Steven Caulker played to this ...,"{'Tottenham Hotspur F.C.': '2009-2013', 'Totte...",Steven Caulker,Q26192,[Steven Roy Caulker],sports,"{""question"": ""Which teams did Steven Caulker p..."
...,...,...,...,...,...,...,...
492,List all political positions Alan Kelly held ...,"{'Leader of the Labour Party': '2020-2022', 'D...",Alan Kelly,Q454245,[Alan Thomas Kelly],political,"{""question"": ""Which political positions did Al..."
493,List all teams Andrés Iniesta played to this ...,"{'FC Barcelona B': '2000-2003', 'Albacete Balo...",Andrés Iniesta,Q43729,[Andrés Iniesta Luján],sports,"{""question"": ""Which teams did Andr\u00e9s Inie..."
494,List all teams Darren Bent played to this day.,"{'Ipswich Town F.C.': '2001-2005, 1998-2001', ...",Darren Bent,Q213459,[Darren Ashley Bent],sports,"{""question"": ""Which teams did Darren Bent play..."
495,List all teams Robert Green played to this day.,"{'Norwich City F.C.': '1999-2006, 1992-1999', ...",Robert Green,Q200770,[Robert Paul Green],sports,"{""question"": ""Which teams did Robert Green pla..."


In [70]:
filtered_time_interval_df.iloc[3]["question"], filtered_time_interval_df.iloc[3]["answers"], filtered_time_interval_df.iloc[3]["time_interval_entries"]


# filtered_temporal_markers_df.iloc[1]["question"], filtered_temporal_markers_df.iloc[1]["answers"], filtered_temporal_markers_df.iloc[1]["temporal_marker_entries"]



('List all teams  Ben May played to this day.',
 {'Millwall F.C.': '2001-2008, 2000-2001',
  'Southampton F.C.': '1998-1999',
  'Colchester United (loan)': '2003, 2004',
  'Fulham F.C.': '1999-2000',
  'Brentford (loan)': '2003-2004, 2004-2005',
  'Scunthorpe United (loan)': '2007',
  'Scunthorpe United F.C.': '2008-2010',
  'Stevenage F.C.': '2010-2012',
  'Barnet (loan)': '2012',
  'Dover Athletic F.C.': '2012-2013',
  'Ebbsfleet United F.C.': '2013-2015',
  'Bromley F.C.': '2015-2016'},
 '{"question": "Which teams did Ben May play for between 2008 and 2015?", "answers": {"Scunthorpe United F.C.": "2008-2010", "Stevenage F.C.": "2010-2012", "Barnet (loan)": "2012", "Dover Athletic F.C.": "2012-2013", "Ebbsfleet United F.C.": "2013-2015"}}')

In [72]:
filtered_temporal_markers_df.iloc[2]["question"], filtered_temporal_markers_df.iloc[2]["answers"], filtered_temporal_markers_df.iloc[2]["temporal_marker_entries"]


('List all political positions  Rob Nicholson held to this day.',
 {'Shadow Minister of Justice Shadow Attorney General of Canada': '2015-2019',
  'Minister of Foreign Affairs': 2015,
  'Minister of National Defence': '2013-2015',
  '[[Minister of Justice (Canada)|Minister of Justice\nAttorney General of Canada]]': '2007-2013',
  'Leader of the Government in the House of Commons': '2006-2007',
  'Minister responsible for Democratic Reform': '2006-2007, 2004-2019, 1984-1993'},
 '{"question": "Which political positions did Rob Nicholson hold after serving as Minister of National Defence?", "answers": {"Shadow Minister of Justice Shadow Attorney General of Canada": "2015-2019", "Minister of Foreign Affairs": "2015"}}')

In [73]:
filtered_temporal_markers_df.to_json("temporal_markers.jsonl", orient="records", lines=True)

In [74]:
filtered_time_interval_df.to_json("time_interval.jsonl", orient="records", lines=True)

In [81]:
filtered_time_interval_df['time_interval_entries'][0]

str

In [82]:
filtered_time_interval_df.head()

,question,answers,subject,wikidata_ID,aliases,questionType,time_interval_entries
0,List all teams John Nutter played to this day.,"{'Wycombe Wanderers F.C.': '2001', 'Blackburn ...",John Nutter,Q6250736,[],sports,"{""question"": ""Which teams did John Nutter play..."
1,List all political positions Brian Pallister ...,"{'22nd Premier of Manitoba': '2016-2021', 'Lea...",Brian Pallister,Q2924976,[Brian William Pallister],political,"{""question"": ""What political positions did Bri..."
2,List all political positions Rob Nicholson he...,{'Shadow Minister of Justice Shadow Attorney G...,Rob Nicholson,Q1566929,[Robert Nicholson],political,"{""question"": ""What political positions did Rob..."
3,List all teams Ben May played to this day.,"{'Millwall F.C.': '2001-2008, 2000-2001', 'Sou...",Ben May,Q2895519,"[Benjamin Steven May, Benjamin May]",sports,"{""question"": ""Which teams did Ben May play for..."
4,List all teams Steven Caulker played to this ...,"{'Tottenham Hotspur F.C.': '2009-2013', 'Totte...",Steven Caulker,Q26192,[Steven Roy Caulker],sports,"{""question"": ""Which teams did Steven Caulker p..."


In [83]:
import json_repair


filtered_time_interval_df['time_interval_entries_parsed'] = filtered_time_interval_df['time_interval_entries'].apply(lambda x: json_repair.repair_json(x))


/var/folders/50/hlflxz7x2t9332_cq2tq_1z40000gn/T/ipykernel_51976/1795166359.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_time_interval_df['time_interval_entries_parsed'] = filtered_time_interval_df['time_interval_entries'].apply(lambda x: json_repair.repair_json(x))


In [91]:
type(filtered_time_interval_df['time_interval_entries_parsed'][0])

str

In [94]:
filtered_time_interval_df['newQuestion'] = filtered_time_interval_df['time_interval_entries_parsed'].apply(lambda x: json.loads(x)['question'])
filtered_time_interval_df['newAnswers'] = filtered_time_interval_df['time_interval_entries_parsed'].apply(lambda x: json.loads(x)['answers'])


filtered_time_interval_df.head()




/var/folders/50/hlflxz7x2t9332_cq2tq_1z40000gn/T/ipykernel_51976/780607269.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_time_interval_df['newQuestion'] = filtered_time_interval_df['time_interval_entries_parsed'].apply(lambda x: json.loads(x)['question'])
/var/folders/50/hlflxz7x2t9332_cq2tq_1z40000gn/T/ipykernel_51976/780607269.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_time_interval_df['newAnswers'] = filtered_time_interval_df['time_interval_entries_parsed'].apply(lambda x

,question,answers,subject,wikidata_ID,aliases,questionType,time_interval_entries,time_interval_entries_parsed,newQuestion,newAnswers
0,List all teams John Nutter played to this day.,"{'Wycombe Wanderers F.C.': '2001', 'Blackburn ...",John Nutter,Q6250736,[],sports,"{""question"": ""Which teams did John Nutter play...","{""question"": ""Which teams did John Nutter play...",Which teams did John Nutter play for between 2...,"{'Stevenage Borough': '2006-2008', 'Gillingham..."
1,List all political positions Brian Pallister ...,"{'22nd Premier of Manitoba': '2016-2021', 'Lea...",Brian Pallister,Q2924976,[Brian William Pallister],political,"{""question"": ""What political positions did Bri...","{""question"": ""What political positions did Bri...",What political positions did Brian Pallister h...,"{'22nd Premier of Manitoba': '2016-2021', 'Lea..."
2,List all political positions Rob Nicholson he...,{'Shadow Minister of Justice Shadow Attorney G...,Rob Nicholson,Q1566929,[Robert Nicholson],political,"{""question"": ""What political positions did Rob...","{""question"": ""What political positions did Rob...",What political positions did Rob Nicholson hol...,{'Minister of Justice Attorney General of Cana...
3,List all teams Ben May played to this day.,"{'Millwall F.C.': '2001-2008, 2000-2001', 'Sou...",Ben May,Q2895519,"[Benjamin Steven May, Benjamin May]",sports,"{""question"": ""Which teams did Ben May play for...","{""question"": ""Which teams did Ben May play for...",Which teams did Ben May play for between 2008 ...,"{'Scunthorpe United F.C.': '2008-2010', 'Steve..."
4,List all teams Steven Caulker played to this ...,"{'Tottenham Hotspur F.C.': '2009-2013', 'Totte...",Steven Caulker,Q26192,[Steven Roy Caulker],sports,"{""question"": ""Which teams did Steven Caulker p...","{""question"": ""Which teams did Steven Caulker p...",Which teams did Steven Caulker play for betwee...,{'England national under-21 football team': '2...


In [95]:
export_data_time_interval = filtered_time_interval_df[['question', 'answers', 'newQuestion', 'newAnswers']]


export_data_time_interval.to_csv("export_data_time_interval.csv", index=False)




In [96]:
# For temporal markers data
temporal_markers_questions = []
temporal_markers_answers = []

for entry in filtered_temporal_markers_df['temporal_marker_entries']:
    entry_dict = json.loads(entry)
    temporal_markers_questions.append(entry_dict['question'])
    temporal_markers_answers.append(entry_dict['answers'])

# Add new columns to filtered_temporal_markers_df
filtered_temporal_markers_df['newQuestion'] = temporal_markers_questions
filtered_temporal_markers_df['newAnswers'] = temporal_markers_answers


export_data_temporal_markers = filtered_temporal_markers_df[['question', 'answers', 'newQuestion', 'newAnswers']]

# Export to CSV
export_data_temporal_markers.to_csv('export_data_temporal_markers.csv', index=False)


/var/folders/50/hlflxz7x2t9332_cq2tq_1z40000gn/T/ipykernel_51976/1480835199.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_temporal_markers_df['newQuestion'] = temporal_markers_questions
/var/folders/50/hlflxz7x2t9332_cq2tq_1z40000gn/T/ipykernel_51976/1480835199.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_temporal_markers_df['newAnswers'] = temporal_markers_answers
